# **Bayesian Optimization을 활용한 딥러닝 하이퍼파라미터 최적화**

## SECOM 반도체 불량 예측 모델

### 학습 목표
1. Bayesian Optimization의 원리 이해 (Surrogate Model, Acquisition Function)
2. 딥러닝 모델의 하이퍼파라미터 최적화 실습
3. Grid Search vs Bayesian Optimization 성능 비교
4. 불균형 데이터셋에서의 모델 성능 향상

### 데이터셋: SECOM (SEmiCOnductor Manufacturing)
- **샘플 수**: 1,567개
- **특성 수**: 590개 (센서 데이터)
- **타겟**: Pass(-1) / Fail(1) - 약 6.6% 불량률
- **특징**: 고차원, 결측치 존재, 심한 클래스 불균형

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필요한 라이브러리 설치 (필요시 주석 해제)
# !pip install optuna torch scikit-learn pandas matplotlib seaborn plotly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    f1_score, precision_score, recall_score, 
    confusion_matrix, classification_report, roc_auc_score
)

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Bayesian Optimization
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate
)

# 재현성을 위한 시드 설정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. 데이터 로딩 및 전처리

In [ ]:
# SECOM 데이터 로딩 (uci-secom.csv)
df = pd.read_csv('./uci-secom.csv')

# Time 컬럼과 Pass/Fail 컬럼 분리
X = df.drop(columns=['Time', 'Pass/Fail'])
y = df['Pass/Fail'].values

# 레이블 변환: -1 -> 0 (Pass), 1 -> 1 (Fail)
y = np.where(y == -1, 0, 1)

print("=" * 50)
print("SECOM 데이터셋 정보")
print("=" * 50)
print(f"데이터 형태: {X.shape}")
print(f"Pass (0): {(y == 0).sum()} ({(y == 0).sum() / len(y) * 100:.2f}%)")
print(f"Fail (1): {(y == 1).sum()} ({(y == 1).sum() / len(y) * 100:.2f}%)")
print(f"결측치 비율: {X.isnull().sum().sum() / X.size * 100:.2f}%")

### 2.1 데이터프레임 기본 정보 확인

In [ ]:
# 데이터프레임 상위 5개 행 확인
print("데이터프레임 상위 5개 행:")
df.head()

In [ ]:
# 데이터프레임 기본 정보
print("데이터프레임 정보:")
print(f"Shape: {df.shape}")
print(f"\n컬럼 타입 분포:")
print(df.dtypes.value_counts())
print(f"\n메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# 기술 통계량 확인
print("기술 통계량 (일부 컬럼):")
df.iloc[:, 1:11].describe()

### 2.2 타겟 변수 (Pass/Fail) 분포 분석

In [ ]:
# 타겟 변수 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 클래스 분포 (원본 레이블)
class_counts = df['Pass/Fail'].value_counts()
colors = ['#2ecc71', '#e74c3c']
labels = ['Pass (-1)', 'Fail (1)']

axes[0].bar(labels, [class_counts[-1], class_counts[1]], color=colors, edgecolor='black')
axes[0].set_ylabel('Count')
axes[0].set_title('Class Distribution (Pass/Fail)')
for i, (label, count) in enumerate(zip(labels, [class_counts[-1], class_counts[1]])):
    axes[0].text(i, count + 20, f'{count}\n({count/len(df)*100:.1f}%)', ha='center', fontsize=11)
axes[0].set_ylim(0, max(class_counts) * 1.15)

# 파이 차트
axes[1].pie([class_counts[-1], class_counts[1]], labels=labels, colors=colors,
            autopct='%1.1f%%', startangle=90, explode=(0, 0.1))
axes[1].set_title('Class Proportion')

plt.tight_layout()
plt.show()

print(f"\n클래스 불균형 비율: 1:{class_counts[-1]/class_counts[1]:.1f} (Pass:Fail)")

### 2.3 결측치 분석

In [ ]:
# 결측치 분석
missing_per_col = X.isnull().sum()
missing_ratio_per_col = X.isnull().sum() / len(X) * 100

# 결측치가 있는 컬럼만 필터링
cols_with_missing = missing_per_col[missing_per_col > 0]

print(f"전체 컬럼 수: {X.shape[1]}")
print(f"결측치가 있는 컬럼 수: {len(cols_with_missing)}")
print(f"결측치가 없는 컬럼 수: {X.shape[1] - len(cols_with_missing)}")
print(f"\n전체 결측치 비율: {X.isnull().sum().sum() / X.size * 100:.2f}%")

# 결측치 비율 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 결측치 비율 히스토그램
axes[0].hist(missing_ratio_per_col, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% threshold')
axes[0].set_xlabel('Missing Ratio (%)')
axes[0].set_ylabel('Number of Columns')
axes[0].set_title('Distribution of Missing Ratios per Column')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 상위 20개 결측치 컬럼
top_missing = missing_ratio_per_col.nlargest(20)
axes[1].barh(range(len(top_missing)), top_missing.values, color='coral', edgecolor='black')
axes[1].set_yticks(range(len(top_missing)))
axes[1].set_yticklabels([f'Feature {i}' for i in top_missing.index])
axes[1].set_xlabel('Missing Ratio (%)')
axes[1].set_title('Top 20 Columns with Highest Missing Ratio')
axes[1].axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 2.4 특성 분포 시각화

In [ ]:
# 일부 특성의 분포 시각화 (결측치가 적은 컬럼 선택)
low_missing_cols = missing_ratio_per_col[missing_ratio_per_col < 5].index[:12].tolist()

fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for idx, col in enumerate(low_missing_cols):
    ax = axes[idx]
    
    # Pass/Fail 별로 분리하여 히스토그램
    pass_data = X.loc[df['Pass/Fail'] == -1, col].dropna()
    fail_data = X.loc[df['Pass/Fail'] == 1, col].dropna()
    
    ax.hist(pass_data, bins=30, alpha=0.6, label='Pass', color='#2ecc71', density=True)
    ax.hist(fail_data, bins=30, alpha=0.6, label='Fail', color='#e74c3c', density=True)
    ax.set_title(f'Feature {col}', fontsize=10)
    ax.legend(fontsize=8)
    ax.tick_params(labelsize=8)

plt.suptitle('Feature Distributions by Class (Pass vs Fail)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 2.5 특성 간 상관관계 분석

In [ ]:
# 상관관계 분석 (일부 특성)
# 결측치가 적은 상위 30개 컬럼 선택
sample_cols = missing_ratio_per_col[missing_ratio_per_col < 5].index[:30].tolist()
corr_matrix = X[sample_cols].corr()

plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={'shrink': 0.8},
            vmin=-1, vmax=1)
plt.title('Correlation Matrix (Sample Features)', fontsize=14)
plt.tight_layout()
plt.show()

# 높은 상관관계를 가진 특성 쌍 찾기
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

print(f"\n높은 상관관계 (|r| > 0.9)를 가진 특성 쌍: {len(high_corr_pairs)}개")
if high_corr_pairs[:5]:
    print("예시:")
    for col1, col2, corr in high_corr_pairs[:5]:
        print(f"  Feature {col1} - Feature {col2}: {corr:.3f}")

### 2.6 EDA 요약

In [ ]:
# EDA 요약
print("=" * 60)
print("EDA 요약")
print("=" * 60)
print(f"\n[데이터 크기]")
print(f"  - 샘플 수: {len(df):,}")
print(f"  - 특성 수: {X.shape[1]}")

print(f"\n[클래스 분포]")
print(f"  - Pass: {class_counts[-1]:,} ({class_counts[-1]/len(df)*100:.1f}%)")
print(f"  - Fail: {class_counts[1]:,} ({class_counts[1]/len(df)*100:.1f}%)")
print(f"  - 불균형 비율: 1:{class_counts[-1]/class_counts[1]:.1f}")

print(f"\n[결측치]")
print(f"  - 결측치 있는 컬럼: {len(cols_with_missing)}개")
print(f"  - 전체 결측치 비율: {X.isnull().sum().sum() / X.size * 100:.2f}%")
print(f"  - 50% 이상 결측 컬럼: {(missing_ratio_per_col >= 50).sum()}개")

print(f"\n[주요 특징]")
print(f"  - 고차원 데이터 (590개 센서 특성)")
print(f"  - 심각한 클래스 불균형 (Fail이 약 6.6%)")
print(f"  - 다수의 결측치 존재 -> 전처리 필요")
print(f"  - 특성 간 높은 상관관계 존재 -> 차원 축소 고려 가능")
print("=" * 60)

### 2.7 데이터 전처리

In [ ]:
# 결측치 비율이 50% 이상인 컬럼 제거
missing_ratio = X.isnull().sum() / len(X)
X_filtered = X.loc[:, missing_ratio < 0.5]
print(f"결측치 필터링 후 특성 수: {X_filtered.shape[1]}")

# 결측치 대치 (평균값)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_filtered)

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print(f"전처리 완료 데이터 형태: {X_scaled.shape}")

In [ ]:
# 학습/검증/테스트 데이터 분할
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=SEED, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=SEED, stratify=y_temp
)

print(f"Train: {X_train.shape[0]} samples (Fail: {y_train.sum()})")
print(f"Val:   {X_val.shape[0]} samples (Fail: {y_val.sum()})")
print(f"Test:  {X_test.shape[0]} samples (Fail: {y_test.sum()})")

In [ ]:
# PyTorch Tensor 변환
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

# 클래스 가중치 계산 (불균형 처리)
n_negative = (y_train == 0).sum()
n_positive = (y_train == 1).sum()
pos_weight = torch.tensor([n_negative / n_positive]).to(device)
print(f"Positive class weight: {pos_weight.item():.2f}")

## 3. 딥러닝 모델 정의

다양한 하이퍼파라미터를 조절할 수 있는 유연한 MLP 모델을 정의합니다.

In [ ]:
class FlexibleMLP(nn.Module):
    """
    유연한 MLP 모델 - Bayesian Optimization을 위한 하이퍼파라미터 조절 가능
    
    Parameters:
    - input_dim: 입력 차원
    - hidden_dims: 은닉층 차원 리스트 (예: [256, 128, 64])
    - dropout_rate: 드롭아웃 비율
    - use_batch_norm: 배치 정규화 사용 여부
    """
    def __init__(self, input_dim, hidden_dims, dropout_rate=0.3, use_batch_norm=True):
        super(FlexibleMLP, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            if use_batch_norm:
                layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        # 출력층
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, 
                lr, batch_size, epochs, pos_weight, early_stopping_patience=10):
    """
    모델 학습 함수 (Early Stopping 포함)
    """
    # DataLoader 생성
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # 손실 함수 및 옵티마이저
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val).squeeze()
            val_preds = (torch.sigmoid(val_outputs) > 0.5).float().cpu().numpy()
            val_f1 = f1_score(y_val.cpu().numpy(), val_preds, zero_division=0)
        
        # Early Stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                break
    
    return best_val_f1

## 4. Bayesian Optimization 이론

### Bayesian Optimization이란?
- **목적**: 평가 비용이 높은 블랙박스 함수의 전역 최적값을 효율적으로 찾는 방법
- **핵심 아이디어**: 이전 평가 결과를 바탕으로 다음에 평가할 지점을 "지능적으로" 선택

### 주요 구성 요소

#### 1. Surrogate Model (대리 모델)
- 실제 목적 함수를 근사하는 확률적 모델
- 주로 **Gaussian Process (GP)** 또는 **Tree-structured Parzen Estimator (TPE)** 사용
- 평균과 불확실성(분산)을 동시에 제공

#### 2. Acquisition Function (획득 함수)
- 다음에 탐색할 지점을 결정하는 함수
- **Exploitation**: 현재까지 좋은 성능을 보인 영역 탐색
- **Exploration**: 불확실성이 높은 영역 탐색
- 대표적인 획득 함수:
  - Expected Improvement (EI)
  - Upper Confidence Bound (UCB)
  - Probability of Improvement (PI)

### Optuna의 TPE (Tree-structured Parzen Estimator)
- 조건부 확률을 활용한 베이지안 최적화 방법
- `p(x|y)` 대신 `p(y|x)`를 모델링
- 고차원 공간에서 효율적

## 5. Optuna를 이용한 Bayesian Optimization 구현

In [ ]:
def objective(trial):
    """
    Optuna Objective Function
    
    최적화할 하이퍼파라미터:
    1. 네트워크 구조: 레이어 수, 각 레이어의 뉴런 수
    2. 정규화: Dropout rate
    3. 학습: Learning rate, Batch size
    """
    
    # 1. 네트워크 구조 하이퍼파라미터
    n_layers = trial.suggest_int('n_layers', 2, 5)
    hidden_dims = []
    for i in range(n_layers):
        hidden_dim = trial.suggest_int(f'hidden_dim_{i}', 32, 512, step=32)
        hidden_dims.append(hidden_dim)
    
    # 2. 정규화 하이퍼파라미터
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    use_batch_norm =  True 

    # 3. 학습 하이퍼파라미터
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    
    # 모델 생성
    model = FlexibleMLP(
        input_dim=X_train.shape[1],
        hidden_dims=hidden_dims,
        dropout_rate=dropout_rate,
        use_batch_norm=use_batch_norm
    ).to(device)
    
    # 모델 학습 및 검증 F1 스코어 반환
    val_f1 = train_model(
        model, X_train_tensor, y_train_tensor,
        X_val_tensor, y_val_tensor,
        lr=lr, batch_size=batch_size, epochs=100,
        pos_weight=pos_weight, early_stopping_patience=15
    )
    
    return val_f1

In [ ]:
# Optuna Study 생성 및 최적화 실행
print("=" * 50)
print("Bayesian Optimization 시작 (Optuna TPE)")
print("=" * 50)

# TPE Sampler 사용 (Bayesian Optimization)
sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction='maximize',  # F1 스코어 최대화
    sampler=sampler,
    study_name='secom_dnn_optimization'
)

# 최적화 실행 (50회 시도)
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "=" * 50)
print("최적화 완료!")
print("=" * 50)
print(f"Best F1 Score: {study.best_value:.4f}")
print(f"\nBest Hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

## 6. 최적화 결과 시각화

In [ ]:
# 1. 최적화 히스토리
fig = plot_optimization_history(study)
fig.update_layout(title="Optimization History (F1 Score)")
fig.show()

In [ ]:
# 2. 하이퍼파라미터 중요도
fig = plot_param_importances(study)
fig.update_layout(title="Hyperparameter Importances")
fig.show()

In [ ]:
# 3. Parallel Coordinate Plot
fig = plot_parallel_coordinate(study)
fig.update_layout(title="Parallel Coordinate Plot")
fig.show()

In [ ]:
# 4. 최적화 과정 시각화 (matplotlib)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Trial별 F1 Score
trials_df = study.trials_dataframe()
ax1 = axes[0, 0]
ax1.plot(trials_df['number'], trials_df['value'], 'b-o', alpha=0.6, markersize=4)
ax1.axhline(y=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.4f}')
ax1.set_xlabel('Trial')
ax1.set_ylabel('F1 Score')
ax1.set_title('Optimization Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Learning Rate 분포
ax2 = axes[0, 1]
ax2.scatter(trials_df['params_lr'], trials_df['value'], c=trials_df['number'], cmap='viridis', alpha=0.6)
ax2.axvline(x=study.best_params['lr'], color='r', linestyle='--', label=f'Best LR: {study.best_params["lr"]:.6f}')
ax2.set_xscale('log')
ax2.set_xlabel('Learning Rate')
ax2.set_ylabel('F1 Score')
ax2.set_title('Learning Rate vs F1 Score')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Dropout Rate 분포
ax3 = axes[1, 0]
ax3.scatter(trials_df['params_dropout_rate'], trials_df['value'], c=trials_df['number'], cmap='viridis', alpha=0.6)
ax3.axvline(x=study.best_params['dropout_rate'], color='r', linestyle='--', label=f'Best Dropout: {study.best_params["dropout_rate"]:.2f}')
ax3.set_xlabel('Dropout Rate')
ax3.set_ylabel('F1 Score')
ax3.set_title('Dropout Rate vs F1 Score')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Number of Layers 분포
ax4 = axes[1, 1]
layer_performance = trials_df.groupby('params_n_layers')['value'].agg(['mean', 'std', 'max'])
ax4.bar(layer_performance.index, layer_performance['mean'], yerr=layer_performance['std'], 
        alpha=0.7, capsize=5, color='steelblue')
ax4.scatter(layer_performance.index, layer_performance['max'], color='red', s=100, zorder=5, label='Best')
ax4.set_xlabel('Number of Layers')
ax4.set_ylabel('F1 Score')
ax4.set_title('Performance by Number of Layers')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 최적 모델로 최종 학습 및 테스트 평가

In [ ]:
# 최적 하이퍼파라미터로 모델 재생성
best_params = study.best_params

# Hidden dimensions 추출
n_layers = best_params['n_layers']
hidden_dims = [best_params[f'hidden_dim_{i}'] for i in range(n_layers)]

print("=" * 50)
print("최적 모델 구성")
print("=" * 50)
print(f"Hidden Layers: {hidden_dims}")
print(f"Dropout Rate: {best_params['dropout_rate']:.3f}")
print(f"Learning Rate: {best_params['lr']:.6f}")
print(f"Batch Size: {best_params['batch_size']}")

In [ ]:
# Train + Validation 데이터로 최종 학습
X_train_full = np.vstack([X_train, X_val])
y_train_full = np.concatenate([y_train, y_val])

X_train_full_tensor = torch.FloatTensor(X_train_full).to(device)
y_train_full_tensor = torch.FloatTensor(y_train_full).to(device)

# 최적 모델 생성
best_model = FlexibleMLP(
    input_dim=X_train.shape[1],
    hidden_dims=hidden_dims,
    dropout_rate=best_params['dropout_rate'],
).to(device)

print(best_model)

In [ ]:
# 최종 학습
train_dataset = TensorDataset(X_train_full_tensor, y_train_full_tensor)
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])

# 학습 히스토리 저장
history = {'train_loss': [], 'train_f1': []}

epochs = 150
for epoch in range(epochs):
    best_model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = best_model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    # 에폭별 메트릭 계산
    best_model.eval()
    with torch.no_grad():
        train_outputs = best_model(X_train_full_tensor).squeeze()
        train_preds = (torch.sigmoid(train_outputs) > 0.5).float().cpu().numpy()
        train_f1 = f1_score(y_train_full, train_preds, zero_division=0)
    
    history['train_loss'].append(epoch_loss / len(train_loader))
    history['train_f1'].append(train_f1)
    
    if (epoch + 1) % 30 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {history['train_loss'][-1]:.4f}, F1: {train_f1:.4f}")

In [ ]:
# 학습 곡선 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], 'b-')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['train_f1'], 'g-')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('F1 Score')
axes[1].set_title('Training F1 Score')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 테스트 세트 평가
best_model.eval()
with torch.no_grad():
    test_outputs = best_model(X_test_tensor).squeeze()
    test_probs = torch.sigmoid(test_outputs).cpu().numpy()
    test_preds = (test_probs > 0.5).astype(int)

print("=" * 50)
print("최종 테스트 결과 (Bayesian Optimization 적용 모델)")
print("=" * 50)
print(f"\nClassification Report:")
print(classification_report(y_test, test_preds, target_names=['Pass', 'Fail']))

print(f"\nMetrics Summary:")
print(f"  F1 Score:    {f1_score(y_test, test_preds):.4f}")
print(f"  Precision:   {precision_score(y_test, test_preds):.4f}")
print(f"  Recall:      {recall_score(y_test, test_preds):.4f}")
print(f"  ROC-AUC:     {roc_auc_score(y_test, test_probs):.4f}")

In [ ]:
# Confusion Matrix 시각화
cm = confusion_matrix(y_test, test_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Pass', 'Fail'],
            yticklabels=['Pass', 'Fail'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Best DNN Model (Bayesian Optimization)')
plt.show()

print(f"\nConfusion Matrix 해석:")
print(f"  True Negative (Pass 정확): {cm[0,0]}")
print(f"  False Positive (Pass->Fail 오분류): {cm[0,1]}")
print(f"  False Negative (Fail->Pass 오분류): {cm[1,0]}")
print(f"  True Positive (Fail 정확): {cm[1,1]}")

## 8. 기본 모델 vs 최적화 모델 비교

In [ ]:
# 기본 하이퍼파라미터 모델 (최적화 전)
baseline_model = FlexibleMLP(
    input_dim=X_train.shape[1],
    hidden_dims=[256, 128, 64],  # 임의의 기본값
    dropout_rate=0.3,
    use_batch_norm=True
).to(device)

# 기본 모델 학습
baseline_dataset = TensorDataset(X_train_full_tensor, y_train_full_tensor)
baseline_loader = DataLoader(baseline_dataset, batch_size=64, shuffle=True)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

for epoch in range(150):
    baseline_model.train()
    for batch_X, batch_y in baseline_loader:
        optimizer.zero_grad()
        outputs = baseline_model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

# 기본 모델 평가
baseline_model.eval()
with torch.no_grad():
    baseline_outputs = baseline_model(X_test_tensor).squeeze()
    baseline_probs = torch.sigmoid(baseline_outputs).cpu().numpy()
    baseline_preds = (baseline_probs > 0.5).astype(int)

print("=" * 50)
print("Baseline 모델 테스트 결과 (최적화 전)")
print("=" * 50)
print(classification_report(y_test, baseline_preds, target_names=['Pass', 'Fail']))

In [ ]:
# 성능 비교 시각화
metrics = ['F1 Score', 'Precision', 'Recall', 'ROC-AUC']

baseline_scores = [
    f1_score(y_test, baseline_preds),
    precision_score(y_test, baseline_preds),
    recall_score(y_test, baseline_preds),
    roc_auc_score(y_test, baseline_probs)
]

optimized_scores = [
    f1_score(y_test, test_preds),
    precision_score(y_test, test_preds),
    recall_score(y_test, test_preds),
    roc_auc_score(y_test, test_probs)
]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, baseline_scores, width, label='Baseline (Default HP)', alpha=0.8)
bars2 = ax.bar(x + width/2, optimized_scores, width, label='Optimized (Bayesian)', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('Baseline vs Bayesian Optimization Model Performance')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

# 값 표시
for bar, score in zip(bars1, baseline_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{score:.3f}', ha='center', va='bottom', fontsize=9)
for bar, score in zip(bars2, optimized_scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# 성능 향상률 계산
print("\n=== 성능 향상률 ===")
for metric, base, opt in zip(metrics, baseline_scores, optimized_scores):
    improvement = (opt - base) / (base + 1e-8) * 100
    print(f"{metric:12s}: {base:.4f} -> {opt:.4f} ({improvement:+.1f}%)")

## 9. 임계값 (Threshold) 튜닝

불균형 데이터에서는 기본 임계값 0.5가 최적이 아닐 수 있습니다.

In [ ]:
# 다양한 임계값에서 성능 측정
thresholds = np.arange(0.1, 0.9, 0.05)
f1_scores_by_threshold = []
precision_by_threshold = []
recall_by_threshold = []

for thresh in thresholds:
    preds_thresh = (test_probs >= thresh).astype(int)
    f1_scores_by_threshold.append(f1_score(y_test, preds_thresh, zero_division=0))
    precision_by_threshold.append(precision_score(y_test, preds_thresh, zero_division=0))
    recall_by_threshold.append(recall_score(y_test, preds_thresh, zero_division=0))

# 최적 임계값 찾기
best_threshold_idx = np.argmax(f1_scores_by_threshold)
best_threshold = thresholds[best_threshold_idx]
best_f1_threshold = f1_scores_by_threshold[best_threshold_idx]

plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores_by_threshold, 'b-o', label='F1 Score', linewidth=2)
plt.plot(thresholds, precision_by_threshold, 'g--', label='Precision', alpha=0.7)
plt.plot(thresholds, recall_by_threshold, 'r--', label='Recall', alpha=0.7)
plt.axvline(x=best_threshold, color='orange', linestyle=':', linewidth=2, 
            label=f'Best Threshold: {best_threshold:.2f}')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Performance Metrics by Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n최적 임계값: {best_threshold:.2f}")
print(f"최적 임계값에서의 F1 Score: {best_f1_threshold:.4f}")

In [ ]:
# 최적 임계값 적용 결과
final_preds = (test_probs >= best_threshold).astype(int)

print("=" * 50)
print(f"최종 결과 (임계값 최적화 적용: {best_threshold:.2f})")
print("=" * 50)
print(classification_report(y_test, final_preds, target_names=['Pass', 'Fail']))

# 최종 Confusion Matrix
cm_final = confusion_matrix(y_test, final_preds)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(confusion_matrix(y_test, test_preds), annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Pass', 'Fail'], yticklabels=['Pass', 'Fail'])
axes[0].set_title('Threshold = 0.5')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

sns.heatmap(cm_final, annot=True, fmt='d', cmap='Oranges', ax=axes[1],
            xticklabels=['Pass', 'Fail'], yticklabels=['Pass', 'Fail'])
axes[1].set_title(f'Threshold = {best_threshold:.2f} (Optimized)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

## 10. 결론 및 요약

In [ ]:
print("=" * 60)
print("   SECOM 불량 예측 - Bayesian Optimization 실습 요약")
print("=" * 60)

print("\n[1] 데이터셋 특성")
print(f"    - 총 샘플: {len(y)} (불량률: {y.mean()*100:.2f}%)")
print(f"    - 특성 수: {X_scaled.shape[1]} (전처리 후)")
print(f"    - 문제점: 극심한 클래스 불균형, 고차원, 결측치")

print("\n[2] Bayesian Optimization 설정")
print(f"    - 방법: Optuna TPE (Tree-structured Parzen Estimator)")
print(f"    - 탐색 횟수: {len(study.trials)} trials")
print(f"    - 최적화 목표: Validation F1 Score 최대화")

print("\n[3] 최적 하이퍼파라미터")
print(f"    - 네트워크 구조: {hidden_dims}")
print(f"    - Dropout Rate: {best_params['dropout_rate']:.3f}")
print(f"    - Learning Rate: {best_params['lr']:.6f}")
print(f"    - Batch Size: {best_params['batch_size']}")

print("\n[4] 최종 성능 (테스트 세트)")
print(f"    - F1 Score: {f1_score(y_test, final_preds):.4f}")
print(f"    - Precision: {precision_score(y_test, final_preds):.4f}")
print(f"    - Recall: {recall_score(y_test, final_preds):.4f}")
print(f"    - ROC-AUC: {roc_auc_score(y_test, test_probs):.4f}")

print("\n[5] 핵심 인사이트")
print("    - Bayesian Optimization은 Grid Search보다 효율적으로 최적 HP 탐색")
print("    - 불균형 데이터에서는 class_weight, pos_weight 조절 필수")
print("    - 임계값 튜닝으로 추가 성능 향상 가능")
print("    - 하이퍼파라미터 중요도 분석으로 영향력 파악")

print("\n" + "=" * 60)

In [ ]:
print("🎉 고생하셨습니다! 🎉")